# Using Python to Perform ETL Processing

In [1]:
# importing the necessary libraries

import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# declaring and assigning the connection variables for the MySQL Server 
# databases with which i'll be working with

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Password"

src_dbname = "HR_management"
dst_dbname = "HR_management_DW2"

In [3]:
# defining the functions for getting data from and setting data into databases

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [4]:
# defining functions for getting data from the setting data into databases

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

# Creating and Populating the Dimension Tables

In [5]:
### extracting the data from the source database tables ###

In [6]:
sql_employees = "SELECT * FROM HR_management.employees;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(2)

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9


In [7]:
sql_jobs = "SELECT * FROM HR_management.jobs;"
df_jobs = get_dataframe(user_id, pwd, host_name, src_dbname, sql_jobs)
df_jobs.head(2)

,job_id,job_title,min_salary,max_salary
0,1,Public Accountant,4200.0,9000.0
1,2,Accounting Manager,8200.0,16000.0


In [8]:
sql_departments = "SELECT * FROM HR_management.departments;"
df_departments = get_dataframe(user_id, pwd, host_name, src_dbname, sql_departments)
df_departments.head(2)

,department_id,department_name,location_id
0,1,Administration,1700
1,2,Marketing,1800


In [9]:
sql_locations = "SELECT * FROM HR_management.locations;"
df_locations = get_dataframe(user_id, pwd, host_name, src_dbname, sql_locations)
df_locations.head(2)

,location_id,street_address,postal_code,city,state_province,country_id
0,1400,2014 Jabberwocky Rd,26192,Southlake,Texas,US
1,1500,2011 Interiors Blvd,99236,South San Francisco,California,US


In [10]:
### performing any necessary transformations ###

In [11]:
# renaming the id's to key's
df_jobs.rename(columns={"job_id":"job_key"}, inplace=True)

df_jobs.head(2)

,job_key,job_title,min_salary,max_salary
0,1,Public Accountant,4200.0,9000.0
1,2,Accounting Manager,8200.0,16000.0


In [12]:
# renaming the id's to key's
df_departments.rename(columns={"department_id":"department_key"}, inplace=True)
df_departments.rename(columns={"location_id":"location_key"}, inplace=True)

df_departments.head(2)

,department_key,department_name,location_key
0,1,Administration,1700
1,2,Marketing,1800


In [13]:
# dropping unecessary columns in the employees table and renaming id's to key's
drop_cols = ['first_name', 'last_name', 'email', 'phone_number']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.rename(columns={"employee_id":"employee_key"}, inplace=True)
df_employees.rename(columns={"job_id":"job_key"}, inplace=True)
df_employees.rename(columns={"department_id":"department_key"}, inplace=True)

df_employees.head(2)

,employee_key,hire_date,job_key,salary,manager_id,department_key
0,100,1987-06-17,4,24000.0,NaN,9
1,101,1989-09-21,5,17000.0,100.0,9


In [14]:
# dropping unecessary columns in the locations table and renaming id's to key's
drop_cols = ['street_address', 'postal_code']
df_locations.drop(drop_cols, axis=1, inplace=True)
df_locations.rename(columns={"location_id":"location_key"}, inplace=True)

df_locations.head(2)

,location_key,city,state_province,country_id
0,1400,Southlake,Texas,US
1,1500,South San Francisco,California,US


In [15]:
### loading the transformed dataframes into the new data warehouse by creating new tables ###

In [16]:
db_operation = "insert"

tables = [('dim_employees', df_employees, 'employee_key'),
          ('dim_jobs', df_jobs, 'job_key'),
          ('dim_departments', df_departments, 'department_key'),
          ('dim_locations', df_locations, 'location_key')]

In [17]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

# Creating and populating the Fact Table

In [18]:
# here I am just looking to see what it looks like using SQL language 
sql_fact = """
SELECT e.employee_id,
e.employee_id,
j.job_id,
d.department_id,
l.location_id,
e.hire_date,
e.salary

FROM HR_management.jobs j
INNER JOIN HR_management.employees e
ON j.job_id = e.job_id
INNER JOIN HR_management.departments d
ON e.department_id = d.department_id
INNER JOIN HR_management.locations l
ON d.location_id = l.location_id ;
"""
df_fact = get_dataframe(user_id, pwd, host_name, src_dbname, sql_fact)
df_fact.head(2)

,employee_id,employee_id,job_id,department_id,location_id,hire_date,salary
0,201,201,10,2,1800,1996-02-17,13000.0
1,202,202,11,2,1800,1997-08-17,6000.0


In [ ]:
### merging the tables to create the fact table ###

In [19]:
df_event_fact = pd.merge(df_jobs, df_employees, on='job_key', how='inner')
df_event_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,manager_id,department_key
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,205.0,11
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,101.0,11


In [20]:
df_event_fact = pd.merge(df_event_fact, df_departments, on='department_key', how='inner')
df_event_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,manager_id,department_key,department_name,location_key
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,205.0,11,Accounting,1700
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,101.0,11,Accounting,1700


In [21]:
df_event_fact = pd.merge(df_event_fact, df_locations, on='location_key', how='inner')
df_event_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,manager_id,department_key,department_name,location_key,city,state_province,country_id
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,205.0,11,Accounting,1700,Seattle,Washington,US
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,101.0,11,Accounting,1700,Seattle,Washington,US


In [22]:
### performing transformations ###

In [23]:
# dropping unecessary columns in the fact table
drop_cols = ['job_title', 'min_salary', 'max_salary', 'department_name', 'manager_id', 'city', 'state_province', 'country_id']
df_event_fact.drop(drop_cols, axis=1, inplace=True)
df_event_fact.head(2)


,job_key,employee_key,hire_date,salary,department_key,location_key
0,1,206,1994-06-07,8300.0,11,1700
1,2,205,1994-06-07,12000.0,11,1700


In [24]:
# reordering columns to make it easier to read 
ordered_cols = ['employee_key', 'job_key', 'department_key', 'location_key', 
                  'hire_date', 'salary']
df_event_fact = df_event_fact[ordered_cols]

In [25]:
df_event_fact.head(3)

,employee_key,job_key,department_key,location_key,hire_date,salary
0,206,1,11,1700,1994-06-07,8300.0
1,205,2,11,1700,1994-06-07,12000.0
2,200,3,1,1700,1987-09-17,4400.0


In [26]:
### writing the dataframe back into the database ###

In [27]:
table_name = 'event_fact'
primary_key = 'employee_key'
db_operation = 'insert'

set_dataframe(user_id, pwd, host_name, dst_dbname, df_event_fact, table_name, primary_key, db_operation)



# Demonstrating that the New Data Warehouse Exists and Contains the Correct Data

In [33]:
# here I am looking to see what the average minimum and maximum salaries are
# for each job that is in my HR database 

# I ordered this aggregation by the avg_min_salary in ascending order to make the viewing/reading
# experience better and easier to follow
sql_test = """
    SELECT dj.job_title AS job_title, 
        AVG(min_salary) AS avg_min_salary, 
        AVG(max_salary) AS avg_max_salary

    FROM HR_management_DW2.event_fact ef
    INNER JOIN HR_management_DW2.dim_employees de
    ON ef.job_key = de.job_key
    INNER JOIN HR_management_DW2.dim_jobs dj
    ON de.job_key = dj.job_key

    GROUP BY dj.job_title
    ORDER BY avg_min_salary ASC;;
    """.format(dst_dbname)


df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)
df_test

,job_title,avg_min_salary,avg_max_salary
0,Stock Clerk,2000.0,5000.0
1,Purchasing Clerk,2500.0,5500.0
2,Shipping Clerk,2500.0,5500.0
3,Administration Assistant,3000.0,6000.0
4,Programmer,4000.0,10000.0
5,Marketing Representative,4000.0,9000.0
6,Human Resources Representative,4000.0,9000.0
7,Accountant,4200.0,9000.0
8,Public Accountant,4200.0,9000.0
9,Public Relations Representative,4500.0,10500.0
